In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Any results you write to the current directory are saved as output.
PATH = "/kaggle/input/m5-forecasting-accuracy/"
print(os.listdir(PATH))

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
['calendar.csv', 'sell_prices.csv', 'sample_submission.csv', 'sales_train_validation.csv']


In [14]:
import os
import gc
import warnings
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
register_matplotlib_converters()
sns.set()

## Calculating Number of Rows in files

In [12]:
for f in os.listdir(PATH):
    with open(f'{PATH}/{f}') as file:
        print("File Name {} => Rows {}".format(f,len(file.readlines())))
        

File Name calendar.csv => Rows 1970
File Name sell_prices.csv => Rows 6841122
File Name sample_submission.csv => Rows 60981
File Name sales_train_validation.csv => Rows 30491


In [23]:
df_temp = pd.read_csv(f"{PATH}/sample_submission.csv",nrows=1)
df_temp.columns

Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object')

## Understanding the Data
#### calendar.csv: 
     columns: ['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI']
       
       calendar.csv shows the dates and various events occur on that date.
       we can use this to relate event with inc or dec in sales or price
       of products.
       
#### sell_prices.csv:
     columns: ['store_id', 'item_id', 'wm_yr_wk', 'sell_price']
     sell_price shows price(sell_price) of each item(item_id) 
     in a particular store(store_id) on a particular day (wm_yr_wk).
     
     This is biggest files of all and can be used to see what effect 
     price of the product have on it's sales.
     
#### sales_train_validation.csv:
     columns:['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913']
       
       sales_train_validation.csv creates a unique id of each prodcuct using 
       its state(state_id),store(store_id),category(cat_id),item(item_id)
       and columns from d_1 to d_1913 shows what number of that particular item
       was sold.
       
#### sample_submission.csv:
     columns:['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
       
       As we need to froecast the number of sales of the items for 28 days,
       so your submission should contain id which is same as id in 
       sales_train_validation and number of item sold in next 28 days(F1 -F28).
       

In [31]:
def downcast_data(df):
    
    #getting columns names with int and float dtypes
    float_cols = df.select_dtypes(include=['float']).columns
    int_cols = df.select_dtypes(include=['int']).columns
    
    #donwcasting the values
    for col in float_cols:
        df[col] = pd.to_numeric(df[col],downcast='float')
    for col in int_cols:
        df[col] = pd.to_numeric(df[col],downcast='integer')
    return df


Downcasting Basically convert a higher bit dtype to lowest possible dtype.<br/>
For more info on to_numeric and downcast see the document [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html)

In [74]:

def loading_and_downcasting():
    calendar = pd.read_csv(f"{PATH}/calendar.csv").pipe(downcast_data)
    sell_price = pd.read_csv(f"{PATH}/sell_prices.csv").pipe(downcast_data)
    sales = pd.read_csv(f"{PATH}/sales_train_validation.csv").pipe(downcast_data)
    submission = pd.read_csv(f"{PATH}/sample_submission.csv").pie(downcast_data)
    print(calendar.shape)
    print(sell_price.shape)
    print(sales.shape)
    print(submission.shape)
    
    return calendar, sell_price, sales, submission

In [ ]:
calendar, sell_price, sales, submission = loading_and_downcasting()

In [54]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [55]:
sell_price.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [56]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,...,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,3,0,0,0,1,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,2,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,1,0,...,5,3,1,0,0,0,1,2,3,0,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,1,1,0,3,1,1,2,1,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


## Label Encoding Caterogical cols

In [57]:
def label_encoder(df,columns):
    
    for col in columns:
        l = LabelEncoder()
        #ignore null values
        not_null = df[col][df[col].notnull()]
        df[col] = pd.Series(l.fit_transform(not_null),index=not_null.index)
        
    return df

In [58]:
calendar = label_encoder(
    calendar, ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
).pipe(downcast_data)

sales = label_encoder(
    sales, ["item_id", "dept_id", "cat_id", "store_id", "state_id"],
).pipe(downcast_data)

sell_price = label_encoder(sell_price, ["item_id", "store_id"]).pipe(downcast_data)

## Melting the sales data for training

In [ ]:
def converting_sales_data(sales):
    id_columns = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]
    
    